In [1]:
import wget
import os
import pandas as pd
import numpy as np
import csv
import urllib.request
from io import StringIO

## Listas de EEUU

En primer lugar accedemos a las urls con las listas de sancionados de EEUU en formato .csv y .xml y las guardamos en local. Cada vez que este proceso es ejecutado, se borran los archivos antiguos en nuestro directorio local y se vuelven a cargar desde la web.

In [2]:
urls = ['https://www.treasury.gov/ofac/downloads/sdn.csv',
        'https://www.treasury.gov/ofac/downloads/add.csv',
        'https://www.treasury.gov/ofac/downloads/sdn.xml']

path = 'UE_EEUU_data_crudo'

for url in urls:
    filename = path + '/' + os.path.basename(url) # get the full path of the file
    if os.path.exists(filename):
        os.remove(filename) # if exist, remove it directly
    wget.download(url, out=filename) # download it to the specific path.

-1 / unknown

A continuación cargamos los .csv con la información de sancionados de EEUU en dos DataFrames relacionados por la columna 'uid', que es la clave primaria del DataFrame df_sdn y la foránea del otro. De estos .csv se seleccionan las columnas relevantes para nuestro estudio. Las columnas de los .csv no vienen etiquetadas con un nombre. Es necesario buscar el nombre correspondiente a cada columna en el .xml, que es el archivo con la información general, y escribirlo.

In [3]:
df_sdn = pd.read_csv('UE_EEUU_data_crudo/sdn.csv',usecols=list(range(3)),header=None).dropna()
df_sdn.columns = ['uid','LastName','sdnType']
df_sdn.head()

,uid,LastName,sdnType
0,36,AEROCARIBBEAN AIRLINES,-0-
1,173,"ANGLO-CARIBBEAN CO., LTD.",-0-
2,306,BANCO NACIONAL DE CUBA,-0-
3,424,BOUTIQUE LA MAISON,-0-
4,475,CASA DE CUBA,-0-


In [4]:
df_add = pd.read_csv('UE_EEUU_data_crudo/add.csv',header=None,usecols=list(range(5))).dropna()
df_add.columns = ['uid','uid_pk_add','Address','City','Country']
df_add.drop(columns=['uid_pk_add'],axis=1,inplace=True)
df_add.head()

,uid,Address,City,Country
0,36,-0-,Havana,Cuba
1,173,"Ibex House, The Minories",London EC3N 1DY,United Kingdom
2,306,Zweierstrasse 35,Zurich CH-8022,Switzerland
3,306,Avenida de Concha Espina 8,Madrid E-28036,Spain
4,306,"Dai-Ichi Bldg. 6th Floor, 10-2 Nihombashi, 2-c...",Tokyo 103,Japan


Se hace inner join a las dos tablas. Como la columna del uid ya no es necesaria despues del join, se puede eliminar. Los valores nulos llevan el valor '-0- ', por lo que se puede sustituir por NaN, que es el nombre del valor nulo para Python. El DataFrame de EEUU con la información relevante queda como:

In [5]:
df_eeuu = pd.merge(df_sdn,df_add,how='inner',on='uid').drop(columns=['uid'],axis=1).replace({'-0- ':np.nan})
df_eeuu.head()

,LastName,sdnType,Address,City,Country
0,AEROCARIBBEAN AIRLINES,NaN,NaN,Havana,Cuba
1,"ANGLO-CARIBBEAN CO., LTD.",NaN,"Ibex House, The Minories",London EC3N 1DY,United Kingdom
2,BANCO NACIONAL DE CUBA,NaN,Zweierstrasse 35,Zurich CH-8022,Switzerland
3,BANCO NACIONAL DE CUBA,NaN,Avenida de Concha Espina 8,Madrid E-28036,Spain
4,BANCO NACIONAL DE CUBA,NaN,"Dai-Ichi Bldg. 6th Floor, 10-2 Nihombashi, 2-c...",Tokyo 103,Japan


In [6]:
df_eeuu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15286 entries, 0 to 15285
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   LastName  15286 non-null  object
 1   sdnType   8103 non-null   object
 2   Address   8250 non-null   object
 3   City      10059 non-null  object
 4   Country   12864 non-null  object
dtypes: object(5)
memory usage: 716.5+ KB


In [7]:
print("La columna con la información del nombre de la entidad es 'LastName'.\n")

print("Se trata de una variable categórica nominal con",
      len(df_eeuu['LastName'].unique()),"valores únicos.\n")

print("Existen",df_eeuu['LastName'].isnull().tolist().count(True),
      "valores nulos de un total de",len(df_eeuu),'valores.')

La columna con la información del nombre de la entidad es 'LastName'.

Se trata de una variable categórica nominal con 10470 valores únicos.

Existen 0 valores nulos de un total de 15286 valores.


In [8]:
print("La columna con la información del tipo de entidad es 'sdnType'.\n")

print("Se trata de una variable categórica nominal con",
      len(df_eeuu['sdnType'].unique()),"valores únicos.\n")

print("Existen",df_eeuu['sdnType'].isnull().tolist().count(True),
      "valores nulos de un total de",len(df_eeuu),'valores.')


La columna con la información del tipo de entidad es 'sdnType'.

Se trata de una variable categórica nominal con 4 valores únicos.

Existen 7183 valores nulos de un total de 15286 valores.


In [9]:
print("Las columnas con la información de la ubicación son 'Address', 'City' y 'Country'.\n")

print("Se tratan de variables categóricas nominales con",
      len(df_eeuu['Address'].unique()),",",len(df_eeuu['City'].unique()),"y",len(df_eeuu['Country'].unique()),
      "valores únicos, respectivamente.\n")

print("Existen",df_eeuu['Address'].isnull().tolist().count(True),
      "valores nulos para 'Address',",df_eeuu['City'].isnull().tolist().count(True),
      "para 'City' y",df_eeuu['Country'].isnull().tolist().count(True),
      "para 'Country' de un total de",len(df_eeuu),"valores.")

Las columnas con la información de la ubicación son 'Address', 'City' y 'Country'.

Se tratan de variables categóricas nominales con 7130 , 3472 y 179 valores únicos, respectivamente.

Existen 7036 valores nulos para 'Address', 5227 para 'City' y 2422 para 'Country' de un total de 15286 valores.


Finalmente, se almacena este DataFrame en formato .csv en la carpeta 'datos_entrada_script_02'.

In [10]:
df_eeuu.to_csv('datos_entrada_script_02/df_eeuu.csv')

## Listas de la UE

Para las listas de la UE, la petición a la URL se hace con un token. No sirve el código utilizado anteriormente porque no acepta el método wget.download. Este proceso no almacena la tabla en local, si no que genera directamente en Python un archivo con los datos. 

In [11]:
url = 'https://webgate.ec.europa.eu/fsd/fsf/public/files/csvFullSanctionsList_1_1/content?token=n008rnnv'
respuesta = urllib.request.urlopen(url)
f = StringIO(bytearray(respuesta.read()).decode())
archivo = csv.reader(f)

A continuación se transforma el archivo generado a DataFrame. El nombre de las columnas aparece en la primera fila. Es necesario poner esa fila como nombre de columna y eliminarla.

In [12]:
df = pd.DataFrame(archivo)
df.rename(columns={0:'column'},inplace=True)
df = df[['column']]
df = df.column.str.split(pat=';',expand=True)

df.columns = df.iloc[0]
df_ue = df.iloc[1:]

In [13]:
df_ue.head()

,﻿fileGenerationDate,Entity_LogicalId,Entity_EU_ReferenceNumber,Entity_UnitedNationId,Entity_DesignationDate,Entity_DesignationDetails,Entity_Remark,Entity_SubjectType,Entity_SubjectType_ClassificationCode,Entity_Regulation_Type,...,Citizenship_Regulation_Programme,Citizenship_Regulation_PublicationUrl,None,None,None,None,None,None,None,None
1,02/08/2022,13,EU.27.28,,,,(UNSC RESOLUTION 1483),P,person,regulation,...,,,None,None,None,None,None,None,None,None
2,02/08/2022,13,EU.27.28,,,,(UNSC RESOLUTION 1483),P,person,regulation,...,,,None,None,None,None,None,None,None,None
3,02/08/2022,13,EU.27.28,,,,(UNSC RESOLUTION 1483),P,person,regulation,...,,,None,None,None,None,None,None,None,None
4,02/08/2022,13,EU.27.28,,,,(UNSC RESOLUTION 1483),P,person,regulation,...,None,None,None,None,None,None,None,None,None,None
5,02/08/2022,13,EU.27.28,,,,(UNSC RESOLUTION 1483),P,person,regulation,...,IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,None,None,None,None,None,None,None,None


Pasamos los datos en crudo de la lista de la UE a local desde el Dataframe. Cada vez que se ejecute el proceso, se elimina el .csv con los datos antiguos y se cargan los nuevos datos.

In [14]:
df_ue.to_csv('UE_EEUU_data_crudo/sancionlist_UE.csv')

Como la lista presenta muchas columnas, el mejor método para identificar cuales de estas son relevantes para nuestro estudio es dividir el DataFrame en grupos de 15 columnas y analizar separadamente las columnas de estas subtablas.

In [15]:
df_ue_0_15 = df_ue.iloc[:,0:15]
df_ue_0_15.head()

,﻿fileGenerationDate,Entity_LogicalId,Entity_EU_ReferenceNumber,Entity_UnitedNationId,Entity_DesignationDate,Entity_DesignationDetails,Entity_Remark,Entity_SubjectType,Entity_SubjectType_ClassificationCode,Entity_Regulation_Type,Entity_Regulation_OrganisationType,Entity_Regulation_PublicationDate,Entity_Regulation_EntryIntoForceDate,Entity_Regulation_NumberTitle,Entity_Regulation_Programme
1,02/08/2022,13,EU.27.28,,,,(UNSC RESOLUTION 1483),P,person,regulation,commission,2003-07-08,2003-07-07,1210/2003 (OJ L169),IRQ
2,02/08/2022,13,EU.27.28,,,,(UNSC RESOLUTION 1483),P,person,regulation,commission,2003-07-08,2003-07-07,1210/2003 (OJ L169),IRQ
3,02/08/2022,13,EU.27.28,,,,(UNSC RESOLUTION 1483),P,person,regulation,commission,2003-07-08,2003-07-07,1210/2003 (OJ L169),IRQ
4,02/08/2022,13,EU.27.28,,,,(UNSC RESOLUTION 1483),P,person,regulation,commission,2003-07-08,2003-07-07,1210/2003 (OJ L169),IRQ
5,02/08/2022,13,EU.27.28,,,,(UNSC RESOLUTION 1483),P,person,regulation,commission,2003-07-08,2003-07-07,1210/2003 (OJ L169),IRQ


In [16]:
df_ue_0_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21786 entries, 1 to 21786
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   ﻿fileGenerationDate                    21786 non-null  object
 1   Entity_LogicalId                       21786 non-null  object
 2   Entity_EU_ReferenceNumber              21786 non-null  object
 3   Entity_UnitedNationId                  21786 non-null  object
 4   Entity_DesignationDate                 21786 non-null  object
 5   Entity_DesignationDetails              21786 non-null  object
 6   Entity_Remark                          21753 non-null  object
 7   Entity_SubjectType                     20795 non-null  object
 8   Entity_SubjectType_ClassificationCode  20767 non-null  object
 9   Entity_Regulation_Type                 20767 non-null  object
 10  Entity_Regulation_OrganisationType     20747 non-null  object
 11  Entity_Regulati

In [17]:
print("La columna con la información del tipo de entidad es 'Entity_SubjectType_ClassificationCode'.\n")

print("Se trata de una variable categórica nominal con",
      len(df_ue['Entity_SubjectType_ClassificationCode'].unique()),"valores únicos.\n")

print("Existen",df_ue['Entity_SubjectType_ClassificationCode'].isnull().tolist().count(True),
      "valores nulos de un total de",len(df_ue),'valores.')

La columna con la información del tipo de entidad es 'Entity_SubjectType_ClassificationCode'.

Se trata de una variable categórica nominal con 16 valores únicos.

Existen 1019 valores nulos de un total de 21786 valores.


In [18]:
df_ue_15_30 = df_ue.iloc[:,15:30]
df_ue_15_30.head()

,Entity_Regulation_PublicationUrl,NameAlias_LastName,NameAlias_FirstName,NameAlias_MiddleName,NameAlias_WholeName,NameAlias_NameLanguage,NameAlias_Gender,NameAlias_Title,NameAlias_Function,NameAlias_LogicalId,NameAlias_RegulationLanguage,NameAlias_Remark,NameAlias_Regulation_Type,NameAlias_Regulation_OrganisationType,NameAlias_Regulation_PublicationDate
1,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,Hussein Al-Tikriti,Saddam,,Saddam Hussein Al-Tikriti,,M,,,17,EN,,regulation,commission,2003-07-08
2,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,,,,Abu Ali,,,,,19,EN,,regulation,commission,2003-07-08
3,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,,,,Abou Ali,FR,,,,380,EN,,regulation,commission,2003-07-08
4,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,,,,,,,,,,,,,,
5,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,,,,,,,,,,,,,,


In [19]:
df_ue_15_30.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21786 entries, 1 to 21786
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Entity_Regulation_PublicationUrl       20724 non-null  object
 1   NameAlias_LastName                     20724 non-null  object
 2   NameAlias_FirstName                    20724 non-null  object
 3   NameAlias_MiddleName                   20724 non-null  object
 4   NameAlias_WholeName                    20724 non-null  object
 5   NameAlias_NameLanguage                 20617 non-null  object
 6   NameAlias_Gender                       20617 non-null  object
 7   NameAlias_Title                        20617 non-null  object
 8   NameAlias_Function                     20587 non-null  object
 9   NameAlias_LogicalId                    19977 non-null  object
 10  NameAlias_RegulationLanguage           19957 non-null  object
 11  NameAlias_Remar

In [20]:
print("La columna con la información del nombre de la entidad es 'NameAlias_WholeName'.\n")

print("Se trata de una variable categórica nominal con",
      len(df_ue['NameAlias_WholeName'].unique()),"valores únicos.\n")

print("Existen",df_ue['NameAlias_WholeName'].isnull().tolist().count(True),
      "valores nulos de un total de",len(df_ue),'valores.')

La columna con la información del nombre de la entidad es 'NameAlias_WholeName'.

Se trata de una variable categórica nominal con 14517 valores únicos.

Existen 1062 valores nulos de un total de 21786 valores.


In [21]:
df_ue_30_45 = df_ue.iloc[:,30:45]
df_ue_30_45.head()

,NameAlias_Regulation_EntryIntoForceDate,NameAlias_Regulation_NumberTitle,NameAlias_Regulation_Programme,NameAlias_Regulation_PublicationUrl,Address_City,Address_Street,Address_PoBox,Address_ZipCode,Address_Region,Address_Place,Address_AsAtListingTime,Address_ContactInfo,Address_CountryIso2Code,Address_CountryDescription,Address_LogicalId
1,2003-07-07,1210/2003 (OJ L169),IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,,,,,,,,,,,
2,2003-07-07,1210/2003 (OJ L169),IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,,,,,,,,,,,
3,2003-07-07,1210/2003 (OJ L169),IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,,,,,,,,,,,
4,,,,,,,,,,,,,,,
5,,,,,,,,,,,,,,,


In [22]:
df_ue_30_45.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21786 entries, 1 to 21786
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   NameAlias_Regulation_EntryIntoForceDate  19883 non-null  object
 1   NameAlias_Regulation_NumberTitle         19882 non-null  object
 2   NameAlias_Regulation_Programme           19876 non-null  object
 3   NameAlias_Regulation_PublicationUrl      19876 non-null  object
 4   Address_City                             19876 non-null  object
 5   Address_Street                           19714 non-null  object
 6   Address_PoBox                            19412 non-null  object
 7   Address_ZipCode                          19411 non-null  object
 8   Address_Region                           19411 non-null  object
 9   Address_Place                            19410 non-null  object
 10  Address_AsAtListingTime                  19399 non-null  o

In [23]:
print("Las columnas con la información de la ubicación son 'Address_City', 'Address_CountryDescription' y 'Address_Street'.\n")

print("Se tratan de variables categóricas nominales con",
      len(df_ue['Address_City'].unique()),',',len(df_ue['Address_CountryDescription'].unique()),'y',
      len(df_ue['Address_Street'].unique()),"valores únicos, respectivamente.\n")

print("Existen",df_ue['Address_City'].isnull().tolist().count(True),
      "valores nulos para 'Address_City',",df_ue['Address_CountryDescription'].isnull().tolist().count(True),
      "para 'Address_CountryDescription y",df_ue['Address_Street'].isnull().tolist().count(True),
      "para 'Address_Street' de un total de",len(df_ue),"valores.")

Las columnas con la información de la ubicación son 'Address_City', 'Address_CountryDescription' y 'Address_Street'.

Se tratan de variables categóricas nominales con 396 , 77 y 542 valores únicos, respectivamente.

Existen 1910 valores nulos para 'Address_City', 2434 para 'Address_CountryDescription y 2072 para 'Address_Street' de un total de 21786 valores.


El DataFrame de la UE con la información relevante queda como:

In [24]:
df_ue = df_ue.loc[:,['Entity_SubjectType_ClassificationCode','NameAlias_WholeName',
                        'Address_City','Address_CountryDescription','Address_Street']]
df_ue.head()

,Entity_SubjectType_ClassificationCode,NameAlias_WholeName,Address_City,Address_CountryDescription,Address_Street
1,person,Saddam Hussein Al-Tikriti,,,
2,person,Abu Ali,,,
3,person,Abou Ali,,,
4,person,,,,
5,person,,,,


Finalmente, se almacena este DataFrame en formato .csv en la carpeta 'datos_entrada_script_02'.

In [25]:
df_ue.to_csv('datos_entrada_script_02/df_ue.csv')